In [21]:
# https://rosettacode.org/wiki/Sorting_algorithms/Insertion_sort#Python

import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import random
import os
import sys
import math
import pickle

## Insertionsort Algorithm

In [22]:
def insertion_sort(L): # x^2
    for i in range(1, len(L)):
        j = i-1 
        key = L[i]
        while j >= 0 and L[j] > key:
           L[j+1] = L[j]
           j -= 1
        L[j+1] = key

## Frequency Count Method

In [23]:
X_y = []
i=50
for n in range(2,i+1): # started with 2 itens until 50 itens in the list
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(insertion_sort)

  # create a list with random number between 1 to 10000
  # with n number in the list
  input = random.choices(range(10000), k=n)
  lp_wrapper(input)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  X_y.append([n, sum(hits)])

dados = np.array(X_y)

X = dados[:, 0]
y = dados[:, 1]
X_reshaped = X.reshape(-1, 1)

In [24]:
resultados_com_menor_loss = []
repeat = 5
registros = []

original_stdout = sys.stdout

with open(os.devnull, 'w') as devnull:
  sys.stdout = devnull

  for i in range(repeat):

    # first combination
    reg1 = PySRRegressor(
      binary_operators=["*", "+"],
      unary_operators=["log", "square", "cube"],
    )

    fit1 = reg1.fit(X_reshaped, y)
    best_program1 = fit1.get_best()

    registro1 = []
    for index, value in enumerate(best_program1):
      registro1.append(value)
      
    registros.append(registro1)

    # second combination
    reg2 = PySRRegressor(
      binary_operators=["*"],
      unary_operators=["log", "square", "cube"],
    )

    fit2 = reg2.fit(X_reshaped, y)
    best_program2 = fit2.get_best()

    registro2 = []
    for index, value in enumerate(best_program2):
      registro2.append(value)
    registros.append(registro2)

    # third combinarion
    reg3 = PySRRegressor(
      binary_operators=["+"],
      unary_operators=["log", "square", "cube"],
    )

    fit3 = reg3.fit(X_reshaped, y)
    best_program3 = fit3.get_best()

    registro3 = []
    for index, value in enumerate(best_program3):
      registro3.append(value)
    registros.append(registro3)
    
sys.stdout = original_stdout

[ Info: Started!
0.0%┣                                              ┫ 0/600 [00:00<00:-6, -0s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           9.001e+05  1.594e+01  y = x₀                                        2           3.822e+04  3.159e+00  y = square(x₀)                                4           1.835e+04  3.670e-01  y = (square(x₀) * 0.74184)                    6           5.538e+03  5.988e-01  y = (square(x₀ + 1.8614) * 0.74184)           8           4.428e+03  1.119e-01  y = ((x₀ + square(x₀ + 1.9549)) * 0.74184)    10          4.382e+03  

In [25]:
registros_ = registros

In [26]:
for i in registros_:
  loss = i[1]
  score = i[2]
  complexity = i[0]
  w = (loss * score)/complexity
  if math.isnan(w):
    i.append(0)
  else:
    i.append(w)

lista_melhor_valor = max(registros_, key=lambda x: x[6])

## Save result

In [27]:
def salvar_dados(dados, arquivo):
  if os.path.exists(arquivo):
    with open(arquivo, 'rb') as f:
      dados_exist = pickle.load(f)

    dados_exist.update(dados)
  else:
    dados_exist = dados
  
  with open(arquivo, 'wb') as f:
    pickle.dump(dados_exist, f)
  
novos_dados = {'insertionsort_N^2': lista_melhor_valor[4]}
caminho_arquivo = 'dados.pickle'

salvar_dados(novos_dados, caminho_arquivo)

In [28]:
def carregar_dados(arquivo):
    # Carrega os dados do arquivo pickle
    with open(arquivo, 'rb') as f:
        dados = pickle.load(f)
    return dados

caminho_arquivo = 'dados.pickle'
dados_carregados = carregar_dados(caminho_arquivo)

print("Conteúdo do arquivo pickle:")
for k, v in dados_carregados.items(): print(k, ' = ',v)

Conteúdo do arquivo pickle:
shellsort_NlogN  =  4.862569*x0*log(x0)
mergesort_NlogN  =  5.5040717*x0 + 8.03698770358715
bubblesort_x^2  =  1.517833856016*x0**2
levenshtein_distance_MN  =  3.52552934090449*x1**2
selectsort_x^2  =  1.13491224207076*x0**2
insertionsort_x^2  =  0.84217347354001*x0**2
